In [1]:
import torch
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
import time
import random

torch.backends.cudnn.deterministic = True

In [2]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 3800
LEARNING_RATE = 1e-4
BATCH_SIZE = 128
NUM_EPOCHS = 15
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
TEXT = data.Field(tokenize='spacy',
                  include_lengths=True) # 为了 packed_padded_sequence
LABEL = data.LabelField(dtype=torch.float)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state=random.seed(RANDOM_SEED),
                                          split_ratio=0.8)

print(f'Num Train: {len(train_data)}')
print(f'Num Valid: {len(valid_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 20000
Num Valid: 5000
Num Test: 25000


In [4]:
TEXT.build_vocab(train_data, max_size=VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 3802
Number of classes: 2


In [5]:
train_loader, valid_loader, test_loader = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    sort_within_batch=True, # 为了 packed_padded_sequence
    device=DEVICE)

In [6]:
def compute_binary_accuracy(model, data_loader, device):
    model.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(data_loader):
            text, text_lengths = batch_data.text
            logits = model(text, text_lengths)
            predicted_labels = (torch.sigmoid(logits) > 0.5).long()
            num_examples += batch_data.label.size(0)
            correct_pred += (predicted_labels == batch_data.label.long()).sum()
        return correct_pred.float()/num_examples * 100

In [7]:
import torch.nn as nn
class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text, text_length):
        embedded = self.embedding(text)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, text_length)
        packed_output, (hidden, cell) = self.lstm(packed)

        return self.fc(hidden.unsqueeze(0)).view(-1)

In [9]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model2 = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(DEVICE)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=LEARNING_RATE)

In [10]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model2.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text, text_lengths = batch_data.text
        
        logits = model2(text, text_lengths)
        cost2 = F.binary_cross_entropy_with_logits(logits, batch_data.label)
        optimizer2.zero_grad()
        
        cost2.backward()
        
        optimizer2.step()
        
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Cost: {cost2:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_binary_accuracy(model2, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_binary_accuracy(model2, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_binary_accuracy(model2, test_loader, DEVICE):.2f}%')

Epoch: 001/015 | Batch 000/157 | Cost: 0.7000
Epoch: 001/015 | Batch 050/157 | Cost: 0.6902
Epoch: 001/015 | Batch 100/157 | Cost: 0.6971
Epoch: 001/015 | Batch 150/157 | Cost: 0.6933
training accuracy: 55.66%
valid accuracy: 55.98%
Time elapsed: 31.34 min
Epoch: 002/015 | Batch 000/157 | Cost: 0.6896
Epoch: 002/015 | Batch 050/157 | Cost: 0.6797
Epoch: 002/015 | Batch 100/157 | Cost: 0.6740
Epoch: 002/015 | Batch 150/157 | Cost: 0.5961
training accuracy: 63.62%
valid accuracy: 63.22%
Time elapsed: 63.40 min
Epoch: 003/015 | Batch 000/157 | Cost: 0.6534
Epoch: 003/015 | Batch 050/157 | Cost: 0.6058
Epoch: 003/015 | Batch 100/157 | Cost: 0.6606
Epoch: 003/015 | Batch 150/157 | Cost: 0.5765
training accuracy: 67.22%
valid accuracy: 67.14%
Time elapsed: 89.73 min
Epoch: 004/015 | Batch 000/157 | Cost: 0.5703
Epoch: 004/015 | Batch 050/157 | Cost: 0.5476
Epoch: 004/015 | Batch 100/157 | Cost: 0.6052
Epoch: 004/015 | Batch 150/157 | Cost: 0.5099
training accuracy: 74.50%
valid accuracy: 73.

KeyboardInterrupt: 

In [4]:
test = torch.randn(1,2,3)
test

tensor([[[ 0.1692, -0.4744,  0.8034],
         [-0.1711, -0.3241, -0.2903]]])

IndexError: index 1 is out of bounds for dimension 0 with size 1